# Standings
We're gonna recreate the standings for the regular season to get a feel for the data <br>
TODO: conferences, goals, home/away record

In [1]:
import pandas as pd
import numpy as np

## Load the correct data

In [2]:
scores = pd.read_pickle('data/scores.pkl')
scores.head()

,Date,Visitor,Visitor Goals,Home,Home Goals,Overtime,Attendance,Length,Notes,Season,Type
0,2019-04-10,Pittsburgh Penguins,3,New York Islanders,4,OT,13917,0 days 03:06:00,at Nassau Veterans Memorial Coliseum (Uniondal...,2018-2019,playoffs
1,2019-04-10,Dallas Stars,3,Nashville Predators,2,NaN,17458,0 days 02:30:00,NaN,2018-2019,playoffs
2,2019-04-10,St. Louis Blues,2,Winnipeg Jets,1,NaN,15321,0 days 02:31:00,NaN,2018-2019,playoffs
3,2019-04-10,Vegas Golden Knights,2,San Jose Sharks,5,NaN,17562,0 days 02:39:00,NaN,2018-2019,playoffs
4,2019-04-10,Columbus Blue Jackets,4,Tampa Bay Lightning,3,NaN,19092,0 days 02:28:00,NaN,2018-2019,playoffs


Let's remove the playoff data for now and group the scores by season.

In [3]:
scores_by_season = scores.loc[scores['Type'] == 'regular season'].groupby('Season')

## Convert scores to statistics

In [4]:
for name, games in scores_by_season:
    # create empty list
    df = pd.DataFrame({'Team': pd.Series([], dtype='category'),
                        'Wins': pd.Series([], dtype='int'),
                        'Losses': pd.Series([], dtype='int'),
                        'OT Wins': pd.Series([], dtype='int'),
                        'OT Losses': pd.Series([], dtype='int')})
    # add teams
    df['Team'] = games['Visitor'].unique()
    # replace nans with zeros
    df = df.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x)
    # count wins, losses and OT
    for _, row in games.iterrows():
        home_idx = df.index[df['Team'] == row['Home']].tolist()[0]
        visit_idx = df.index[df['Team'] == row['Visitor']].tolist()[0]
        if row['Home Goals'] > row['Visitor Goals']:    # Home team wins
            df.at[home_idx, 'Wins'] += 1
            if pd.notnull(row['Overtime']):   # 2OT, etc possible
                df.at[home_idx, 'OT Wins'] += 1
                df.at[visit_idx, 'OT Losses'] += 1
            else:
                df.at[visit_idx, 'Losses'] += 1
        elif row['Home Goals'] < row['Visitor Goals']:  # Visitor team wins
            df.at[visit_idx, 'Wins'] += 1
            if pd.notnull(row['Overtime']):   # 2OT, etc possible
                df.at[visit_idx, 'OT Wins'] += 1
                df.at[home_idx, 'OT Losses'] += 1
            else:
                df.at[home_idx, 'Losses'] += 1
    # add metrics
    df['GP'] = df.sum(axis=1, numeric_only=True)
    df['Points'] = 2 * df['Wins'] + 1 * df['OT Losses']
    df['Reg Win'] = df['Wins'] - df['OT Wins']
    # print table
    print('Season ' + name)
    display(df.sort_values('Points', ascending=False))
    print()

Season 2018-2019


,Team,Wins,Losses,OT Wins,OT Losses,GP,Points,Reg Win
29,Tampa Bay Lightning,62.0,16.0,13.0,4.0,95.0,128.0,49.0
3,Boston Bruins,49.0,24.0,11.0,9.0,93.0,107.0,38.0
2,Calgary Flames,50.0,25.0,5.0,7.0,87.0,107.0,45.0
10,Washington Capitals,48.0,26.0,9.0,8.0,91.0,104.0,39.0
4,New York Islanders,48.0,27.0,11.0,7.0,93.0,103.0,37.0
14,San Jose Sharks,46.0,27.0,8.0,9.0,90.0,101.0,38.0
8,Nashville Predators,47.0,29.0,9.0,6.0,91.0,100.0,38.0
21,Toronto Maple Leafs,46.0,28.0,6.0,8.0,88.0,100.0,40.0
27,Pittsburgh Penguins,44.0,26.0,7.0,12.0,89.0,100.0,37.0
11,Winnipeg Jets,47.0,30.0,9.0,5.0,91.0,99.0,38.0



Season 2019-2020


,Team,Wins,Losses,OT Wins,OT Losses,GP,Points,Reg Win
7,Boston Bruins,44.0,14.0,6.0,12.0,76.0,100.0,38.0
23,St. Louis Blues,42.0,19.0,9.0,10.0,80.0,94.0,33.0
30,Colorado Avalanche,42.0,20.0,5.0,8.0,75.0,92.0,37.0
17,Tampa Bay Lightning,43.0,21.0,8.0,6.0,78.0,92.0,35.0
1,Washington Capitals,41.0,20.0,10.0,8.0,79.0,90.0,31.0
29,Philadelphia Flyers,41.0,21.0,10.0,7.0,79.0,89.0,31.0
28,Pittsburgh Penguins,40.0,23.0,11.0,6.0,80.0,86.0,29.0
14,Vegas Golden Knights,39.0,24.0,9.0,8.0,80.0,86.0,30.0
25,Edmonton Oilers,37.0,25.0,6.0,9.0,77.0,83.0,31.0
21,Dallas Stars,37.0,24.0,11.0,8.0,80.0,82.0,26.0



Season 2020-2021


,Team,Wins,Losses,OT Wins,OT Losses,GP,Points,Reg Win
23,Vegas Golden Knights,32.0,11.0,7.0,2.0,52.0,66.0,25.0
18,Colorado Avalanche,30.0,9.0,3.0,4.0,46.0,64.0,27.0
29,Florida Panthers,29.0,12.0,8.0,5.0,54.0,63.0,21.0
7,Carolina Hurricanes,29.0,10.0,8.0,5.0,52.0,63.0,21.0
22,Tampa Bay Lightning,30.0,13.0,6.0,2.0,51.0,62.0,24.0
6,Washington Capitals,29.0,13.0,5.0,4.0,51.0,62.0,24.0
14,Toronto Maple Leafs,28.0,12.0,6.0,5.0,51.0,61.0,22.0
11,New York Islanders,28.0,13.0,8.0,4.0,53.0,60.0,20.0
2,Pittsburgh Penguins,28.0,14.0,7.0,3.0,52.0,59.0,21.0
8,Minnesota Wild,28.0,13.0,5.0,3.0,49.0,59.0,23.0
